# Assignment 4: Multilingual BERT and Zero-Shot Transfer (10 Marks)

## Due: 31 March 2022

Welcome to the 4th and last assignment of the course. In this assignment we will learn how to fine-tune a multilingual BERT or mBERT model on a Natural Language Inference task [XNLI](https://arxiv.org/abs/1809.05053). We will fine-tune the model on English Training data and then evaluate the performance of the fine-tuned models on different languages demonstrating the zero-shot capabilities of mBERT. 

In [123]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    data_dir = "/content/gdrive/MyDrive/data/xnli"
except:
    data_dir = "/datadrive/t-kabir/work/repos/PlakshaNLP/source/Assignment4/data/xnli"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [124]:
# Install required libraries
!pip install numpy
!pip install pandas
!pip install torch
!pip install tqdm
!pip install matplotlib
!pip install transformers
!pip install tqdm

In [125]:
# We start by importing libraries that we will be making use of in the assignment.
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import copy
import tqdm


## XNLI: Task Description

XNLI is a multilingual benchmark for Natural Language Inference, that contains training data available in English which was obtained from the popular [MNLI](https://cims.nyu.edu/~sbowman/multinli/), and test and dev sets available for 15 different languages. In NLI, we are given two sentences, one is a premise and other an hypothesis, and the task is to predict whether the hypothesis is i) entialed in the premise, or ii) contradicts the premise, or iii) neutral to the premise. 

<img src="https://i.ibb.co/bd4P20K/nli-examples.jpg" alt="nli-examples" border="0">

This makes NLI a multi-class classification task where we want to predict the correct label out of the three possible classes. We start by loading the dataset into memory. The training set in XNLI is comparitively huge with around 400k examples, which can lead to higher training times. Hence for the purpose of this assignment we will work with a fraction of the full data i.e. ~40k examples

In [126]:
def load_xnli_dataset(lang, split = "train"):
    filename = os.path.join(data_dir, f"{split}-{lang}.tsv")
    sentence1s = []
    sentence2s = []
    labels = []
    with open(filename) as f:
        for i,line in enumerate(f):
            if i == 0:
                continue
            row = line.split("\t")
            sentence1 = row[0]
            sentence2 = row[1]
            label = row[2].split("\n")[0]
            sentence1s.append(sentence1)
            sentence2s.append(sentence2)
            labels.append((label))
    
    return pd.DataFrame({
        "premise": sentence1s,
        "hypothesis" : sentence2s,
        "label" : labels
    })

In [127]:
# Load Training data in english
train_en_data = load_xnli_dataset("en", "train")[:40000]

#Like last assignment we will use split the training data to get some validation examples as well
train_en_data, val_en_data = train_test_split(train_en_data, test_size=0.05)

print(f"Number of examples in training data: {len(train_en_data)}")
print(f"Number of examples in validation data: {len(val_en_data)}")

train_en_data.head()

Number of examples in training data: 38000
Number of examples in validation data: 2000


,premise,hypothesis,label
38965,is it uh i i believe is it sugar cane or i 'm ...,I believe it 's sugar cane but I 'm not sure,entailment
39006,You 're a bit of a dream . ',You are reality in my head .,contradiction
37308,Is there anything cuter than that ?,That is so ugly .,contradiction
37996,okay well um thanks for hearing me ramp i gues...,The conversation was pleasant even though I ra...,neutral
24109,It is this package that allows a manufacturer ...,The manufacturing plant was small but the prod...,neutral


In [128]:
# Load Test data in other languages
test_langs = ["ar", "bg", "de", "el", "en", "es", "fr", "hi", "ru", "sw", "th", "tr", "ur", "vi", "zh"]

lang2test_df = {lang : load_xnli_dataset(lang, "dev") for lang in test_langs}

In [129]:
print(f"Number of Test examples: {len(lang2test_df['en'])}")
lang2test_df["en"].head()

Number of Test examples: 2489


,premise,hypothesis,label
0,"And he said, Mama, I'm home.",He didn't say a word.,contradiction
1,"And he said, Mama, I'm home.",He told his mom he had gotten home.,entailment
2,I didn't know what I was going for or anything...,I have never been to Washington so when I was ...,neutral
3,I didn't know what I was going for or anything...,I knew exactly what I needed to do as I marche...,contradiction
4,I didn't know what I was going for or anything...,I was not quite certain what I was going to do...,entailment


In [130]:
for lang, test_df in lang2test_df.items():
    print(f"{lang} test set:")
    print(test_df.head())
    print("***************************\n")

ar test set:
                                             premise  \
0                        وقال، ماما، لقد عدت للمنزل.   
1                        وقال، ماما، لقد عدت للمنزل.   
2  لم أعرف من أجل ماذا أنا ذاهب أو أي شىْ ، لذلك ...   
3  لم أعرف من أجل ماذا أنا ذاهب أو أي شىْ ، لذلك ...   
4  لم أعرف من أجل ماذا أنا ذاهب أو أي شىْ ، لذلك ...   

                                          hypothesis          label  
0                                  لم ينطق ببنت شفة.  contradiction  
1                        أخبر أمه أنه قد عاد للمنزل.     entailment  
2  لم أذهب إلى واشنطن من قبل، لذا عندما تم تكليفي...        neutral  
3  لقد عرفت بالضبط ما الذي احتجت أن أفعله عندما م...  contradiction  
4  لم أكن متأكدًا مما سأفعله لذلك ذهبت إلى واشنطن...     entailment  
***************************

bg test set:
                                             premise  \
0                      И той каза: Мамо, у дома съм.   
1                      И той каза: Мамо, у дома съм.   
2  Не знаех за какво

## mBERT using HuggingFace's transformers library

mBERT is a multilingual variant of BERT, which is trained on wikipedia articles in around [100 languages](BertTokenizer). Like monolingual BERT the transformers library also provides pre-trained models and tokenizers for multilingual BERT. To create an instance of one, we only need to specify `"bert-base-multilingual-cased"` or `"bert-base-multilingual-uncased"` in `BertTokenizer.from_pretrained` and `BertModel.from_pretrained` methods and that's it! See examples below for a demonstration:

In [131]:
from transformers import BertTokenizer, BertModel

In [132]:
mbert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [133]:
mbert_tokenizer.tokenize("thinking machines")

['thinking', 'machines']

In [134]:
mbert_tokenizer.tokenize("maquinas de pensar")

['maquinas', 'de', 'pensar']

In [135]:
mbert_tokenizer.tokenize("सोच मशीन")

['स', '##ो', '##च', 'म', '##शी', '##न']

As you can see mBERT's tokenizer works on different languages. We can similarly load a pretrained mbert model and feed data in different languages

In [136]:
mbert_model = BertModel.from_pretrained("bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [137]:
mbert_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

As you can see the architecture is identical to the original BERT model. The only thing that is different is the shape of word_embeddings which is 105879 X 768, meaning there are 105879 unique tokens supported by mBERT (uncased). In contrast BERT (uncased) supports 30522 tokens.

In [138]:
en_sent = "thinking machines"
tokenizer_output = mbert_tokenizer(en_sent, return_tensors="pt")
input_ids, attn_mask = tokenizer_output["input_ids"], tokenizer_output["attention_mask"]

mbert_model(input_ids, attention_mask = attn_mask)

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0295,  0.0307,  0.0245,  ...,  0.0328, -0.0562,  0.0789],
                                                        [ 0.2827,  0.4737, -0.1378,  ..., -0.1892,  0.1408, -0.3370],
                                                        [ 0.1871,  0.6193,  0.1692,  ...,  0.0987, -0.0519, -0.0380],
                                                        [-0.0354,  0.4805, -0.2533,  ...,  0.7390,  0.1286, -0.6764]]],
                                                      grad_fn=<NativeLayerNormBackward0>)),
                                              ('pooler_output',
                                               tensor([[ 7.3179e-02,  1.0201e-01,  1.0094e-01,  9.1636e-02,  1.4111e-01,
                                                         3.3841e-01,  9.0549e-02, -6.1100e-02, -1.3902e-01,  2.4607e-01,
                                                        -1.7

In [139]:
es_sent = "maquinas de pensar"
tokenizer_output = mbert_tokenizer(es_sent, return_tensors="pt")
input_ids, attn_mask = tokenizer_output["input_ids"], tokenizer_output["attention_mask"]

mbert_model(input_ids, attention_mask = attn_mask)

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0657, -0.0614,  0.0060,  ..., -0.0240, -0.0420, -0.0673],
                                                        [ 0.2430,  0.5252, -0.0694,  ..., -0.0108,  0.5518, -0.4204],
                                                        [-0.2587, -0.2086,  0.1115,  ..., -0.7734,  0.1412, -0.9133],
                                                        [-0.1032,  0.7808,  0.1022,  ...,  0.0870,  0.1456, -0.5586],
                                                        [-0.0765,  0.3234, -0.2513,  ...,  0.3950,  0.2862, -1.1907]]],
                                                      grad_fn=<NativeLayerNormBackward0>)),
                                              ('pooler_output',
                                               tensor([[ 1.4176e-01,  3.4401e-02,  1.5948e-01,  1.7631e-01,  1.8552e-01,
                                                         4.2113

In [140]:
hi_sent = "सोच मशीन"
tokenizer_output = mbert_tokenizer(hi_sent, return_tensors="pt")
input_ids, attn_mask = tokenizer_output["input_ids"], tokenizer_output["attention_mask"]

mbert_model(input_ids, attention_mask = attn_mask)

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0183,  0.0577,  0.1038,  ..., -0.0349, -0.0897, -0.0034],
                                                        [-0.2319,  0.0178, -0.0228,  ..., -0.4928,  0.1712, -0.3489],
                                                        [-0.1214,  0.0804,  0.2790,  ..., -0.0647, -0.2073, -0.6718],
                                                        ...,
                                                        [-0.0835,  0.4992, -0.1798,  ...,  0.5467, -0.4407,  0.0024],
                                                        [ 0.1764,  0.7496,  0.2607,  ...,  0.2637, -0.2617, -0.2716],
                                                        [-0.3781,  0.5859,  0.2654,  ...,  0.3965, -0.5680, -0.8965]]],
                                                      grad_fn=<NativeLayerNormBackward0>)),
                                              ('pooler_output',
     

Hence, we can very easily use mBERT for generating predictions on texts written in different languages.

## Task 1: Fine-tune mBERT on XNLI

We can now start fine-tuning mBERT on this dataset. We will start by defining the custom `Dataset` class for the task and then define the model and training loop.

## Task 1.1: Custom Dataset Class (2 Marks)

Like in the previous assignments, implement the `XNLImBertDataset` class below that processes and stores the data as well as provides a way to iterate through the dataset. The details about various methods in the class are mentioned in their docstrings

In [141]:
class XNLImBertDataset(Dataset):
    
    def __init__(self, premises,
                 hypotheses,
                 labels,
                 max_length,
                mbert_variant = "bert-base-multilingual-uncased"):
        
        """
        Constructor for the `XNLImBertDataset` class. Stores the `premises`, `hypotheses` and `labels`
        which can then be used by other methods. Also initializes the tokenizer.
        
        Inputs:
            - premises (list) : A list of sentences constituting the premise in each example
            - hypotheses (list) : A list of sentences constituting the hypothesis in each example
            - labels (list) : A list of labels denoting for each premise-hypothesis pair.
            - max_length (int): Maximum length of the encoded sequence.  
                                If number of tokens are lower than `max_length` add padding otherwise truncate
        
        
        Note that labels are in the form of strings "entailment", "contradiction" and "neutral". For training the
        models we will want the labels in the numeric form, so you should define a mapping from the text label
        to a numeric id. You should order the labels in alphabetical order while defining the mapping i.e. 
        contadiction -> 0, entailment -> 1, "neutral" - > 2 (such that we have consistency across everyone) 
        
        """
        
        self.premises = None
        self.hypotheses = None
        self.labels = [0]*len(labels)
        self.max_length = max_length
        self.tokenizer = None
        self.label2id = {'contradiction':0,'entailment':1,'neutral':2} # Define it as a dictionary
        
        # YOUR CODE HERE
        self.premises=premises
        self.hypotheses=hypotheses
        self.tokenizer = BertTokenizer.from_pretrained(mbert_variant)
        
        for i in range(len(self.premises)):
          self.labels[i]=self.label2id[labels[i]]
        #raise NotImplementedError()
        
    def __len__(self):
        """
        Returns the length of the dataset
        """
        length = None
        
        # YOUR CODE HERE
        #raise NotImplementedError()
        length=len( self.labels )
        
        return length
    
    def __getitem__(self, idx):
        """
        
        Returns the features and label corresponding to the the `idx` entry in the dataset.
        
        Inputs:
            - idx (int): Index corresponding to the sentence_pair,label to be returned
        
        Returns:
            - input_ids (torch.tensor): Indices of the tokens in the sentence pair.
                                        Shape of the tensor should be (`seq_len`,)
            - mask (torch.tensor): Attention mask indicating which tokens are padded.
            - label (int): Label for the premise-hypothesis pair
            
        Hint: We have 2 sentences in a pair which must be concatenated using the [SEP] token before we tokenize and encode them
        
        """
        
        input_ids = None
        mask = None
        label = None
        
        # YOUR CODE HERE
        sentences=[0]*len(self.labels)
        
        for i in range(len(self.premises)):
          sentences[i]=self.premises[i]+"[SEP]"+ self.hypotheses[i]

        
        tokenizer_output=self.tokenizer(sentences[idx],max_length=self.max_length, padding="max_length", truncation = True, return_tensors="pt")


        mask=tokenizer_output["attention_mask"]
        input_ids=tokenizer_output["input_ids"]
        label=self.labels[idx]
        #raise NotImplementedError()
        
        return input_ids.squeeze(0), mask.squeeze(0), label

In [142]:
print("Running Sample Test Cases")
sample_premises = ["A man inspects the uniform of a figure in some East Asian country.",
                    "An older and younger man smiling.",
                   "A soccer game with multiple males playing."
                    ]
sample_hypotheses = ["The man is sleeping.",
                     "Two men are smiling and laughing at the cats playing on the floor.",
                    "Some men are playing a sport."]
sample_labels = ["contradiction", "neutral", "entailment"]
sample_max_len = 32
sample_dataset = XNLImBertDataset(
    sample_premises,
    sample_hypotheses,
    sample_labels,
    sample_max_len
)
print(f"Sample Test Case 1: Checking if `__len__` is implemented correctly")
dataset_len= len(sample_dataset)
expected_len = len(sample_labels)
print(f"Dataset Length: {dataset_len}")
print(f"Expected Length: {expected_len}")
assert len(sample_dataset) == len(sample_premises)
print("Sample Test Case Passed!")
print("****************************************\n")

print(f"Sample Test Case 2: Checking if `__getitem__` is implemented correctly for `idx= 0`")
sample_idx = 0
input_ids, mask, label = sample_dataset.__getitem__(sample_idx)
expected_input_ids =  torch.tensor([  101,   143, 10564, 15450, 84789, 10107, 10103, 38884, 10108,   143,
        16745, 10104, 10970, 11344, 17147, 11913,   119,   102, 10103, 10564,
        10127, 55860,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0])
expected_mask = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0])
expected_label = 0
print(f"input_ids:\n {input_ids}")
print(f"Expected input_ids:\n {expected_input_ids}")
assert (expected_input_ids == input_ids).all()

print(f"mask:\n {mask}")
print(f"Expected mask:\n {expected_mask}")
assert (expected_mask == mask).all()

print(f"label:\n {label}")
print(f"Expected label:\n {expected_label}")
assert expected_label == label

print("Sample Test Case Passed!")
print("****************************************\n")

print(f"Sample Test Case 3: Checking if `__getitem__` is implemented correctly for `idx= 1`")
sample_idx = 1
input_ids, mask, label = sample_dataset.__getitem__(sample_idx)
expected_input_ids = torch.tensor([  101, 10144, 18585, 10110, 24392, 10564, 14965, 64581,   119,   102,
        10536, 10562, 10320, 14965, 64581, 10110, 18418, 82863, 10160, 10103,
        45670, 14734, 10125, 10103, 21005,   119,   102,     0,     0,     0,
            0,     0])
expected_mask = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 0])
expected_label = 2
print(f"input_ids:\n {input_ids}")
print(f"Expected input_ids:\n {expected_input_ids}")
assert (expected_input_ids == input_ids).all()

print(f"mask:\n {mask}")
print(f"Expected mask:\n {expected_mask}")
assert (expected_mask == mask).all()

print(f"label:\n {label}")
print(f"Expected label:\n {expected_label}")
assert expected_label == label

print("Sample Test Case Passed!")
print("****************************************\n")


print(f"Sample Test Case 4: Checking if `__getitem__` is implemented correctly for `idx= 2`")
sample_idx = 2
input_ids, mask, label = sample_dataset.__getitem__(sample_idx)
expected_input_ids = torch.tensor([  101,   143, 20071, 11336, 10171, 18248, 19592, 14734,   119,   102,
        10970, 10562, 10320, 14734,   143, 13148,   119,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0])
expected_mask = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
expected_label = 1
print(f"input_ids:\n {input_ids}")
print(f"Expected input_ids:\n {expected_input_ids}")
assert (expected_input_ids == input_ids).all()

print(f"mask:\n {mask}")
print(f"Expected mask:\n {expected_mask}")
assert (expected_mask == mask).all()

print(f"label:\n {label}")
print(f"Expected label:\n {expected_label}")
assert expected_label == label

print("Sample Test Case Passed!")
print("****************************************\n")



sample_premises = ["एक आदमी किसी पूर्वी एशियाई देश में एक आकृति की वर्दी का निरीक्षण करता है।",
                    "एक बूढ़ा और छोटा आदमी मुस्कुरा रहा है।",
                   "एक फ़ुटबॉल खेल जिसमें कई पुरुष खेल रहे हैं।"
                    ]
sample_sentence2s = ["आदमी सो रहा है।",
                     "फर्श पर खेल रही बिल्लियों को देखकर दो आदमी मुस्कुरा रहे हैं और हंस रहे हैं।",
                    "कुछ पुरुष कोई खेल खेल रहे हैं।"
                    ]
sample_labels = ["contradiction", "neutral", "entailment"]
sample_max_len = 36
sample_dataset = XNLImBertDataset(
    sample_premises,
    sample_sentence2s,
    sample_labels,
    sample_max_len
)

print(f"Sample Test Case 5: Checking for hindi")
sample_idx = 1
input_ids, mask, label = sample_dataset.__getitem__(sample_idx)
expected_input_ids =  torch.tensor([  101, 11384,   569, 30119, 10949, 11142, 74535, 10949,   533, 13764,
        25695,   571, 12114, 19086, 10949, 36335,   580,   591,   102,   568,
        11551, 17109, 12334, 56426, 52061,   569, 28393, 41790, 20106, 11483,
        91329, 19086, 29931,   533, 13764,   102])
expected_mask = torch.tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
expected_label = 2
print(f"input_ids:\n {input_ids}")
print(f"Expected input_ids:\n {expected_input_ids}")
assert (expected_input_ids == input_ids).all()

print(f"mask:\n {mask}")
print(f"Expected mask:\n {expected_mask}")
assert (expected_mask == mask).all()

print(f"label:\n {label}")
print(f"Expected label:\n {expected_label}")
assert expected_label == label

print("Sample Test Case Passed!")
print("****************************************\n")




Running Sample Test Cases
Sample Test Case 1: Checking if `__len__` is implemented correctly
Dataset Length: 3
Expected Length: 3
Sample Test Case Passed!
****************************************

Sample Test Case 2: Checking if `__getitem__` is implemented correctly for `idx= 0`
input_ids:
 tensor([  101,   143, 10564, 15450, 84789, 10107, 10103, 38884, 10108,   143,
        16745, 10104, 10970, 11344, 17147, 11913,   119,   102, 10103, 10564,
        10127, 55860,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0])
Expected input_ids:
 tensor([  101,   143, 10564, 15450, 84789, 10107, 10103, 38884, 10108,   143,
        16745, 10104, 10970, 11344, 17147, 11913,   119,   102, 10103, 10564,
        10127, 55860,   119,   102,     0,     0,     0,     0,     0,     0,
            0,     0])
mask:
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0])
Expected mask:
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Initialize dataset and dataloaders for english training and validation sets

In [143]:
max_seq_len = 128
batch_size = 8

train_en_premises, train_en_hypotheses = train_en_data["premise"].values, train_en_data["hypothesis"].values
train_en_labels = train_en_data["label"].values

val_en_premises, val_en_hypotheses = val_en_data["premise"].values, val_en_data["hypothesis"].values
val_en_labels = val_en_data["label"].values

train_en_dataset = XNLImBertDataset(train_en_premises, train_en_hypotheses, train_en_labels, max_seq_len)
val_en_dataset = XNLImBertDataset(val_en_premises, val_en_hypotheses, val_en_labels, max_seq_len)

train_en_dataloader = DataLoader(train_en_dataset, batch_size = batch_size)
val_en_dataloader = DataLoader(val_en_dataset, batch_size = batch_size)

## Task 1.2: Implement mBERT Based Classifier for NLI (2 Marks)

Similar to last assignment implement a classifier with an mBERT module followed by a classification layer. Note that unlike last time we have 3 classes now, so we can no longer use Sigmoid function in the output layer and instead will need to use the Softmax function. You can refer [here](https://cs231n.github.io/linear-classify/#softmax-classifier) if you need a primer on how the softmax function works. Hence, this time instead of getting a single output from the model for an input, denoting the probability of the poistive class, we will get 3 numbers as output for each input denoting the probability of each of the 3 classes. Also, it is common to use Log of the Softmax function instead of plain softmax to obtain log-probabilities. Log-Softmax is numerically more stable and hence it is often more used in practice. You can read about it's usage in pytorch [here](https://pytorch.org/docs/stable/generated/torch.nn.LogSoftmax.html). Implement the `mBERTNLIClassifierModel` below

In [144]:

class mBERTNLIClassifierModel(nn.Module):
    
    def __init__(self, d_hidden = 768, mbert_variant = "bert-base-multilingual-uncased"):
        
        """
        Constructor for the `mBERTNLIClassifierModel` class. Use this to define  the network architecture
        which should be: Input -> mBERT -> Linear Layer -> Log-Softmax
        
        Inputs:
            - d_hidden (int): Size of the hidden representations of mbert
            - mbert_variant (str): mBERT variant to use
        
        """
        super(mBERTNLIClassifierModel, self).__init__()
        
        self.mbert_layer = None
        self.output_layer = None
        self.log_softmax_layer = None
        
        # YOUR CODE HERE
        self.mbert_layer=BertModel.from_pretrained(mbert_variant)
        self.output_layer= nn.Linear(d_hidden, 3)
        self.log_softmax_layer = nn.LogSoftmax(dim=1)
        #raise NotImplementedError()
        
        
    def forward(self, input_ids, attn_mask):
        
        """
        Forward Passes the inputs through the network and obtains the prediction
        
        Inputs:
            - input_ids (torch.tensor): A torch tensor of shape [batch_size, seq_len]
                                        representing the sequence of token ids
            - attn_mask (torch.tensor): A torch tensor of shape [batch_size, seq_len]
                                        representing the attention mask such that padded tokens are 0 and rest 1
                                        
        Returns:
          - output (torch.tensor): A torch tensor of shape [batch_size, 3] containing (log) probabilities
          of each class 
                                                
        """
        
        output = None
        
        # YOUR CODE HERE
        o=  self.mbert_layer(input_ids, attention_mask = attn_mask)
  
        output = self.log_softmax_layer (self.output_layer(o.pooler_output))
        #raise NotImplementedError()
        
        return output

In [145]:
print(f"Running Sample Test Cases!")
torch.manual_seed(42)
model = mBERTNLIClassifierModel()

sample_premises = ["A man inspects the uniform of a figure in some East Asian country.",
                    "An older and younger man smiling.",
                   "A soccer game with multiple males playing."
                    ]
sample_hypotheses = ["The man is sleeping.",
                     "Two men are smiling and laughing at the cats playing on the floor.",
                    "Some men are playing a sport."]
sample_labels = ["contradiction", "neutral", "entailment"]
sample_max_len = 32
sample_dataset = XNLImBertDataset(
    sample_premises,
    sample_hypotheses,
    sample_labels,
    sample_max_len
)


print("Sample Test Case 1")
sample_idx = 0
input_ids, attn_mask, label = sample_dataset.__getitem__(sample_idx)
mbert_cls_out = model(input_ids.unsqueeze(0), attn_mask.unsqueeze(0)).detach().numpy()
expected_mbert_cls_out = np.array([[-0.9885041, -1.479876,  -0.915788 ]])
print(f"Model Output: {mbert_cls_out }")
print(f"Expected Output: {expected_mbert_cls_out}")

assert mbert_cls_out .shape == expected_mbert_cls_out.shape
assert np.allclose(mbert_cls_out, expected_mbert_cls_out, 1e-4)
print("Test Case Passed! :)")
print("******************************\n")

print("Sample Test Case 2")
sample_idx = 1
input_ids, attn_mask, label = sample_dataset.__getitem__(sample_idx)
mbert_cls_out = model(input_ids.unsqueeze(0), attn_mask.unsqueeze(0)).detach().numpy()
expected_mbert_cls_out = np.array([[-0.97441876, -1.4775381,  -0.9304163 ]])
print(f"Model Output: {mbert_cls_out }")
print(f"Expected Output: {expected_mbert_cls_out}")

assert mbert_cls_out .shape == expected_mbert_cls_out.shape
assert np.allclose(mbert_cls_out, expected_mbert_cls_out, 1e-4)
print("Test Case Passed! :)")
print("******************************\n")


Running Sample Test Cases!


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sample Test Case 1
Model Output: [[-0.9885042 -1.479876  -0.9157878]]
Expected Output: [[-0.9885041 -1.479876  -0.915788 ]]
Test Case Passed! :)
******************************

Sample Test Case 2
Model Output: [[-0.9744187  -1.4775381  -0.93041617]]
Expected Output: [[-0.97441876 -1.4775381  -0.9304163 ]]
Test Case Passed! :)
******************************



## Task 1.3: Training and Evaluating the Model (4 Marks)

Similar to previous assignments implement the `train` and `evaluate` functions below. There will be though a couple of differences this time. First, for training the model we can no longer use Binary Cross Entropy Loss because as the name suggests it is applicable for binary classification problems. Instead we will use the [Negative Log-Likelihood Loss function](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) instead. Second, while evaluating the accuracy we can no longer use a threshold to convert the probabilities into the labels, since we will now have 3 probability values instead of a single one, corresponding to the each class. In such cases it is common to predict the class as the label which has the highest probability (or equivalently log probability).


In [146]:
def get_accuracy(pred_labels, act_labels):
  l = len(pred_labels)
  #print(l)
  lst= []
 
  for i in range(l):
    if (pred_labels[i]==act_labels[i]):
      lst.append(1)
    else:
      lst.append(0)

  accuracy = sum(lst)/l
  
  # YOUR CODE HERE
  #raise NotImplementedError()

  return accuracy

In [147]:
def convert_probs_to_labels(pred_probs):
  
  labels=torch.argmax(torch.tensor(pred_probs), dim=1)
  # YOUR CODE HERE
  #raise NotImplementedError()

  return labels

In [148]:
def evaluate(model, test_dataloader,device="cuda"):
    
    """
    Evaluates `model` on test dataset

    Inputs:
        - model (mBERTNLIClassifierModel): mBERT based classifier model to be evaluated
        - test_dataloader (torch.utils.DataLoader): A dataloader defined over the test dataset

    Returns:
        - accuracy (float): Average accuracy over the test dataset 
    """
    
    
    accuracy = None
    
    # YOUR CODE HERE
    model.to(device)
    model.eval()
   
    accuracy = 0
    #raise NotImplementedError()
    with torch.no_grad():
      for test_batch in test_dataloader:
        input_ids, mask, labels = test_batch
        input_ids = input_ids.to(device)
        mask=mask.to(device)
        #labels =labels.to(device)
        #print(labels)
        pred_probs = model(input_ids, mask)
        pred_probs=pred_probs
        #print(pred_probs)
        #pred_probs=pred_probs.to(device)
        pred_probs = pred_probs.detach().cpu().numpy()
        input_ids =  input_ids.detach().cpu().numpy()
        batch_accuracy = get_accuracy(convert_probs_to_labels(pred_probs), labels)
        accuracy+=batch_accuracy

    
    return accuracy/len(test_dataloader)
    
    
    
def train(model, train_dataloader, val_dataloader,
          lr = 1e-5, num_epochs = 3,device="cuda"
          ):
    
    """
    Runs the training loop. Define the loss function as NLLLoss
    and optimizer as Adam and train for `num_epochs` epochs.

    Inputs:
        - model (mBERTNLIClassifierModel): mBERT based classifer model to be trained
        - train_dataloader (torch.utils.DataLoader): A dataloader defined over the training dataset
        - val_dataloader (torch.utils.DataLoader): A dataloader defined over the validation dataset
        - lr (float): The learning rate for the optimizer
        - num_epochs (int): Number of epochs to train the model for.
        - device (str): Device to train the model on. Can be either 'cuda' (for using gpu) or 'cpu'

    Returns:
        - best_model (mBERTNLIClassifierModel): model corresponding to the highest validation accuracy (checked at the end of each epoch)
        - best_val_accuracy (float): Validation accuracy corresponding to the best epoch
    """
    model = model.to(device)
        
    best_val_accuracy = float("-inf")
    best_model = None
    
    # YOUR CODE HERE
    loss_fn = torch.nn.NLLLoss()
    optimizer = Adam(model.parameters(), lr)
    #raise NotImplementedError()
    for epoch in range(num_epochs):
        #model.train() # Since we are evaluating model at the end of every epoch, it is important to bring it back to train mode
        epoch_loss = 0
        
        # 2. Write Training Loop (store the loss for each batch in epoch_loss like done in previous assignments)
        # YOUR CODE HERE
        for train_batch in tqdm.tqdm(train_dataloader):
            optimizer.zero_grad()
            #print(train_batch)
            input_ids, mask, labels= train_batch
            
            input_ids = input_ids.to(device)
            mask=mask.to(device)
            labels = labels.to(device)
            preds = model(input_ids, mask)
            
            loss = loss_fn(preds, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
       
        
        epoch_loss = epoch_loss / len(train_dataloader)
        
        # 3. Evaluate on validation data by calling `evaluate` and store the validation accuracy in `val_accurracy`
        val_accuracy = 0
        # YOUR CODE HERE
        val_accuracy =evaluate(model, val_dataloader)
        
        
        # Model selection
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = copy.deepcopy(model) # Create a copy of model
        
        print(f"Epoch {epoch} completed | Average Training Loss: {epoch_loss} | Validation Accuracy: {val_accuracy}")
 
    best_model.zero_grad()
    return best_model, best_val_accuracy

In [149]:
torch.manual_seed(42)
print("Training on 100 data points for sanity check")

max_seq_len = 128
batch_size = 8

sample_premises, sample_hypotheses = train_en_data["premise"].values[:100], train_en_data["hypothesis"].values[:100]
sample_labels = train_en_data["label"].values[:100]

sample_dataset = XNLImBertDataset(sample_premises, sample_hypotheses, sample_labels, max_seq_len)
sample_dataloader = DataLoader(sample_dataset, batch_size = batch_size)


model = mBERTNLIClassifierModel()
best_model, best_val_acc = train(model, sample_dataloader, sample_dataloader, lr = 5e-5, num_epochs = 10, device="cuda")
print(f"Best Validation Accuracy: {best_val_acc}")
print(f"Expected Best Validation Accuracy: {0.99}")

Training on 100 data points for sanity check


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 13/13 [00:01<00:00,  7.03it/s]


Epoch 0 completed | Average Training Loss: 1.1184610861998339 | Validation Accuracy: 0.375


100%|██████████| 13/13 [00:01<00:00,  6.99it/s]


Epoch 1 completed | Average Training Loss: 1.089647889137268 | Validation Accuracy: 0.5480769230769231


...

Epoch 7 completed | Average Training Loss: 0.16816660331992003 | Validation Accuracy: 0.9903846153846154


100%|██████████| 13/13 [00:01<00:00,  7.07it/s]


Epoch 8 completed | Average Training Loss: 0.03179901849048642 | Validation Accuracy: 0.9807692307692307


100%|██████████| 13/13 [00:01<00:00,  7.11it/s]


Epoch 9 completed | Average Training Loss: 0.08652137474228556 | Validation Accuracy: 1.0
Best Validation Accuracy: 1.0
Expected Best Validation Accuracy: 0.99


Since we just trained and evaluated on same 100 examples, you should expect nearly perfect 99% accuracy. Now let's train on the entire dataset.

In [150]:
model = mBERTNLIClassifierModel()
best_model, best_val_acc = train(model, train_en_dataloader, val_en_dataloader, lr = 1e-5, num_epochs = 2)
print(f"Best Validation Accuracy: {best_val_acc}")
print(f"Expected Best Validation Accuracy: {0.7675}")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4750/4750 [27:21<00:00,  2.89it/s]


Epoch 0 completed | Average Training Loss: 0.7354497155578513 | Validation Accuracy: 0.7185


100%|██████████| 4750/4750 [27:10<00:00,  2.91it/s]


Epoch 1 completed | Average Training Loss: 0.41100476534645025 | Validation Accuracy: 0.725
Best Validation Accuracy: 0.725
Expected Best Validation Accuracy: 0.7675


## Task 1.4: Zero-Shot Transfer (2 Marks)

Pre-trained multilingual models like mBERT have shown to exhibit zero-shot transfer capabilities to new langauges for which the model was never fine-tuned on. You can read more about zero-shot transfer in mBERT in this [paper](https://arxiv.org/abs/1906.01502). We now test this phenomenon for ourselves, where we will evaluate the performance of the mBERT classifier that we just trained on the English on the test sets in 15 different languages. Implement the `evaluate_on_diff_langs` function below that does that

In [157]:
def evaluate_on_diff_langs(model, lang2test_df, max_length = 128, batch_size = 8, device = "cuda"):
    
    """
    Evaluates the accuracy of the fine-tuned model on test data in different langauges.
    
    Inputs:
        - model (mBERTNLIClassifierModel): mBERT based classifer model fine-tuned on English data
        - lang2test_df (dict): A dictionary with langauges as keys and
                                their corresponding test sets (in form of pandas dataframe)
                                as values
                                
    Returns:
        - lang2acc (dict): A dictionary with language ids as keys and the accuracy on it's test set as values
                            eg: {"en" : 0.8, "fr" : 0.77, "hi": 0.72, ...}
    
    """
    
    lang2acc = {}
    
    # YOUR CODE HERE
    model.eval()
    model = model.to(device)
   
    language_values=list(lang2test_df.values())
  
    language=list(lang2test_df.keys())
    
   
    for i in range(len(language_values)):
      test_data=language_values[i]
      test_premises, test_hypotheses = test_data["premise"].values, test_data["hypothesis"].values
      test_labels = test_data["label"].values  
      test_dataset = XNLImBertDataset(test_premises, test_hypotheses, test_labels, max_length)
    
      test_dataloader = DataLoader( test_dataset, batch_size = batch_size)
      #lang_acc=evaluate(model, test_dataloader, device = "cuda")
      accuracy=evaluate(model, test_dataloader,device="cuda")
      
      lang2acc[language[i]]=accuracy
    return lang2acc  
   

    
    
    #lang2acc = evaluate(model, test_en_dataloader, device = "cpu")

    #raise NotImplementedError()
    
    

In [158]:






lang2acc = evaluate_on_diff_langs(best_model, lang2test_df, max_length = 128, batch_size = 8, device = "cuda")
expected_vals = {'ar': 0.5989583333333334,
 'bg': 0.6454326923076923,
 'de': 0.6698717948717948,
 'el': 0.6402243589743589,
 'en': 0.7263621794871795,
 'es': 0.6923076923076923,
 'fr': 0.6802884615384616,
 'hi': 0.5893429487179487,
 'ru': 0.6478365384615384,
 'sw': 0.53125,
 'th': 0.35136217948717946,
 'tr': 0.610176282051282,
 'ur': 0.5637019230769231,
 'vi': 0.6193910256410257,
 'zh': 0.6073717948717948}
print(f"Langauge to Accuracy:\n {lang2acc}")
print(f"Expected Values:\n {expected_vals}")

Langauge to Accuracy:
 {'ar': 0.5665064102564102, 'bg': 0.610176282051282, 'de': 0.6538461538461539, 'el': 0.5913461538461539, 'en': 0.7139423076923077, 'es': 0.6658653846153846, 'fr': 0.6510416666666666, 'hi': 0.5480769230769231, 'ru': 0.6105769230769231, 'sw': 0.5132211538461539, 'th': 0.35096153846153844, 'tr': 0.5981570512820513, 'ur': 0.5356570512820513, 'vi': 0.5933493589743589, 'zh': 0.5805288461538461}
Expected Values:
 {'ar': 0.5989583333333334, 'bg': 0.6454326923076923, 'de': 0.6698717948717948, 'el': 0.6402243589743589, 'en': 0.7263621794871795, 'es': 0.6923076923076923, 'fr': 0.6802884615384616, 'hi': 0.5893429487179487, 'ru': 0.6478365384615384, 'sw': 0.53125, 'th': 0.35136217948717946, 'tr': 0.610176282051282, 'ur': 0.5637019230769231, 'vi': 0.6193910256410257, 'zh': 0.6073717948717948}


Don't worry if the values do not match exactly, but you can expect similar patterns i.e. the fine-tuned model on English data, performs reasonably on other new langauges as well compared to it's performance on English test data. Performance on langauges like German, French and Spanish is much closer to the performance on English. However, it is on the lower side for languages like Swahilli, Urdu and Thai. The values are still surprisingly high, considering a random guess will fetch you an accuracy of 33%.